📊 Desafio 46 — Previsão de Churn em Academia

📂 Dataset: dia_46_churn_academia.csv
Este conjunto de dados contém informações de clientes de uma academia, incluindo frequência semanal, satisfação, plano contratado e se houve cancelamento (churn).

🔍 Tarefas:

Qual é a taxa geral de churn?

Há diferença de churn entre os tipos de plano (Mensal, Trimestral, Anual)?

Os clientes com menor tempo médio de treino têm maior churn?

Qual a média de satisfação entre clientes que saíram e os que permaneceram?

Treine um modelo de classificação para prever o Churn. Sugestões:

Use LogisticRegression ou RandomForestClassifier.

Aplique train_test_split, depois avalie com accuracy_score e classification_report.

Quais variáveis mais influenciam o modelo?

In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [31]:
df = pd.read_csv("dia_46_churn_academia.csv")
df.head()

,Cliente,Sexo,Idade,Dias_Por_Semana,Tempo_Medio_Treino,Nota_Satisfacao,Plano,Churn
0,Cliente_1,Masculino,31,1,69.1,3,Anual,1
1,Cliente_2,Feminino,35,6,53.3,1,Mensal,0
2,Cliente_3,Feminino,37,1,79.7,4,Trimestral,1
3,Cliente_4,Masculino,39,5,65.9,5,Trimestral,1
4,Cliente_5,Masculino,23,3,45.3,2,Anual,1


In [32]:
#Qual é a taxa geral de churn?
taxaChurn = df["Churn"].value_counts(normalize=True) * 100
print(taxaChurn)

Churn
1    70.0
0    30.0
Name: proportion, dtype: float64


In [33]:
#Há diferença de churn entre os tipos de plano (Mensal, Trimestral, Anual)?
churnPlano = df.groupby("Plano")["Churn"].value_counts(normalize=True).unstack().round(2) * 100
print(churnPlano)

Churn          0     1
Plano                 
Anual       36.0  64.0
Mensal      22.0  78.0
Trimestral  36.0  64.0


In [47]:
#Os clientes com menor tempo médio de treino têm maior churn?
df["GrupoTreino"] = pd.qcut(df["Tempo_Medio_Treino"], 4)
churnTempo = df.groupby("GrupoTreino", observed=True)["Churn"].value_counts(normalize=True).unstack()
print("\nChurn por tempo médio de treino:\n", churnTempo)


Churn por tempo médio de treino:
 Churn                    0         1
GrupoTreino                         
(19.999, 40.975]  0.230000  0.770000
(40.975, 51.7]    0.326733  0.673267
(51.7, 62.225]    0.323232  0.676768
(62.225, 87.8]    0.320000  0.680000


In [56]:
#Qual a média de satisfação entre clientes que saíram e os que permaneceram?
mediaSatisfacaoChurn = df.groupby("Churn")["Nota_Satisfacao"].mean()
print(mediaSatisfacaoChurn)

Churn
0    4.058333
1    2.685714
Name: Nota_Satisfacao, dtype: float64


In [60]:
#Treine um modelo de classificação para prever o Churn.
df_modelo = df.copy()
le = LabelEncoder()
df_modelo["Sexo"] = le.fit_transform(df_modelo["Sexo"])
df_modelo["Plano"] = le.fit_transform(df_modelo["Plano"])


X = df_modelo.drop(["Cliente", "Churn", "GrupoTreino"], axis=1)
y = df_modelo["Churn"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

modelo = RandomForestClassifier(random_state=42)
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)

print(f"Acurácia: {accuracy_score(y_test, y_pred)}")

Acurácia: 0.7666666666666667


In [62]:
#Quais variáveis mais influenciam o modelo?
importancias = pd.DataFrame({"Variável": X.columns, "Importância": modelo.feature_importances_}).sort_values(by="Importância", ascending=False)
print(importancias)

             Variável  Importância
4     Nota_Satisfacao     0.269146
3  Tempo_Medio_Treino     0.237907
1               Idade     0.202966
2     Dias_Por_Semana     0.186224
5               Plano     0.069220
0                Sexo     0.034538
